# Demo MultiContainer with ShiftAmount Activity
In certain scenarios it is important to distinguish which types of objects are available in a container. HasContainer does not provide this capability, while MultiContainer do. A MultiContainer represents a named set of containers. The number of allowed containers is limited by parameter **store_capacity**.
For each container a name (**id_** property), a **capacity** and **level** is specified. 
A MultiContainer can be initialized with parameter **initials** as provided in the example for site and vessel configuration below.

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Site with MultiContainer
The clas HasContainer has now been replaced with HasMultiContainer. The **from_site** can contain four different container types, but right now contains only two: One called MP with a capacity of 10 and a level of 2 and one called TP with a capacity of 2 and a level of 0.

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasMultiContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5d",  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 2, "capacity": 10},
        {"id": "TP", "level": 0, "capacity": 2},
    ],  # Capacity of the hopper - "Beunvolume"
}  # The actual volume of the site
from_site = Site(**data_from_site)

## Creation of Vessel with MultiContainer
The class ContainerDependentMovable has been replaced with MultiContainerDependentMovable. The vessel has two containers, one for MPs and one for TPs, each with a capacity of two and a current level of zero.

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.MultiContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
    ),
    {},
)

# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 0, "capacity": 2},
        {"id": "TP", "level": 0, "capacity": 2},
    ],  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
}

hopper = TransportProcessingResource(**data_hopper)

## ShiftAmount Activity for MultiContainer
The **amount** specifies the objects to be transfered and the **id_** parameter specifies to which container this activity relates.

In [4]:
shift_amount_activity_loading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer MP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 1,
    "id_":"MP",
    "duration": 20,
    "postpone_start": False,
}
activity = model.ShiftAmountActivity(**shift_amount_activity_loading_data)


## Run simulation

In [5]:
my_env.run()

In [7]:
display(plot.get_log_dataframe(hopper, [activity]))
display(plot.get_log_dataframe(activity, [activity]))

,Timestamp,Activity,ActivityState,geometry,container level,container level MP,container level TP
0,1970-01-01 00:00:00,Transfer MP,START,POINT (4.18055556 52.18664444),0,0,0
1,1970-01-01 00:00:00,Transfer MP,START,POINT (4.18055556 52.18664444),0,0,0
2,1970-01-01 00:00:20,Transfer MP,STOP,POINT (4.18055556 52.18664444),0,1,0
3,1970-01-01 00:00:20,Transfer MP,STOP,POINT (4.18055556 52.18664444),0,1,0


,Timestamp,Activity,ActivityState
0,1970-01-01 00:00:00,Transfer MP,START
1,1970-01-01 00:00:20,Transfer MP,STOP
